<h1>Notebook di prova per Altair</h1>

<h2>Caricamento delle librerie e il dataset</h2>

In [1]:
#!pip install vega

  Using cached pandas-1.5.3-cp39-cp39-win_amd64.whl (10.9 MB)


  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4


<p>Import delle librerie necessarie</p>

In [2]:
import pandas as pd
import altair as alt
import altair_viewer as av
from altair import datum
import re
import string
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import word_tokenize

<p>carico il dataset</p>

In [5]:
df = pd.read_csv('datasetForAltair-eng.csv', index_col=0, low_memory=False)

In [6]:
df.head(3)

,user,date,likeCount,tweetText,hashtags,lang,replies,retweetCount,replyCount,Year,Month,Day,WeekDay,hour,minutes,dayofYear,date_only,tweetText_proc,Sentiment
0,jdemay,2023-02-05 23:59:16+00:00,0,Planning to use #ChatGPT to help you with high...,['ChatGPT'],en,0.0,0.0,NaN,2023,2,5,Sunday,23,59,36,2023-02-05,planning use help high school math ? maybe thi...,pos
1,CarlosSilvaB81,2023-02-05 23:58:51+00:00,3,"@Carmtans @stkirsch Chat gpt, is nothing but a...",NaN,en,0.0,0.0,NaN,2023,2,5,Sunday,23,58,36,2023-02-05,"chat gpt , nothing super search engine , progr...",neg
2,saadnajeebsaad,2023-02-05 23:58:15+00:00,0,#WeatherUpdate #Karachi #Khi #TempUpdate #Sunr...,"['WeatherUpdate', 'Karachi', 'Khi', 'TempUpdat...",en,0.0,0.0,NaN,2023,2,5,Sunday,23,58,36,2023-02-05,sunrise : 07:12 sunset : 06:19 pm current temp...,neu


<p>stampo info relative al dataset corrente</p>

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287309 entries, 0 to 287308
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user            280601 non-null  object 
 1   date            287309 non-null  object 
 2   likeCount       287309 non-null  int64  
 3   tweetText       287309 non-null  object 
 4   hashtags        177869 non-null  object 
 5   lang            287309 non-null  object 
 6   replies         281713 non-null  float64
 7   retweetCount    287202 non-null  float64
 8   replyCount      5596 non-null    float64
 9   Year            287309 non-null  int64  
 10  Month           287309 non-null  int64  
 11  Day             287309 non-null  int64  
 12  WeekDay         287309 non-null  object 
 13  hour            287309 non-null  int64  
 14  minutes         287309 non-null  int64  
 15  dayofYear       287309 non-null  int64  
 16  date_only       287309 non-null  object 
 17  tweetText_

<h2>Processing dei dati testuali</h2>

In [8]:
def pre_process(text):
    if isinstance(text, str):
        # Remove links
        text = re.sub(r'http\S+', '', text)

        # Remove mentions and hashtags
        text = re.sub(r'@\w+|#\w+', '', text)

        # Remove punctuation marks
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word.lower() not in stop_words]

        # Remove empty and short words
        tokens = [word for word in tokens if len(word) > 2]

        # Join the tokens back into a single string
        cleaned_text = ' '.join(tokens)

        return cleaned_text
    else:
        return ''

In [9]:
#applico la pulizia del testo
df['tweetText_proc'] = df['tweetText_proc'].apply(pre_process)

In [11]:
df.head(3)

,user,date,likeCount,tweetText,hashtags,lang,replies,retweetCount,replyCount,Year,Month,Day,WeekDay,hour,minutes,dayofYear,date_only,tweetText_proc,Sentiment
0,jdemay,2023-02-05 23:59:16+00:00,0,Planning to use #ChatGPT to help you with high...,['ChatGPT'],en,0.0,0.0,NaN,2023,2,5,Sunday,23,59,36,2023-02-05,planning use help high school math maybe think...,pos
1,CarlosSilvaB81,2023-02-05 23:58:51+00:00,3,"@Carmtans @stkirsch Chat gpt, is nothing but a...",NaN,en,0.0,0.0,NaN,2023,2,5,Sunday,23,58,36,2023-02-05,chat gpt nothing super search engine programme...,neg
2,saadnajeebsaad,2023-02-05 23:58:15+00:00,0,#WeatherUpdate #Karachi #Khi #TempUpdate #Sunr...,"['WeatherUpdate', 'Karachi', 'Khi', 'TempUpdat...",en,0.0,0.0,NaN,2023,2,5,Sunday,23,58,36,2023-02-05,sunrise 0712 sunset 0619 current temp humidity...,neu


<h2>Analisi del sentimento attraverso i mesi nei commenti di twitter</h2>

In [13]:
# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Group by month and sentiment
df_grouped = df.groupby([df['date'].dt.to_period('M'), 'Sentiment']).size().unstack()

# Convert index to string and extract month names
df_grouped.index = df_grouped.index.strftime('%B %Y')

# Reshape the dataframe for Altair
df_grouped = df_grouped.reset_index().melt('date', var_name='Sentiment', value_name='Value')

# Define color domain and range
domain = ['pos', 'neg', 'neu']
range_ = ['green', 'red', 'blue']

# Create the Altair chart
chart = alt.Chart(df_grouped, title=alt.Title(
       "Twitter's sentiment values over months",
       subtitle="Visualizing sentiment values over months in the Twitter dataset"
   )).mark_line().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Month', format='%B %Y', tickCount=12)),
    y=alt.Y('Value:Q', axis=alt.Axis(title='Value')),
    opacity=alt.value(0.9),
    color=alt.Color('Sentiment',
                    title="Sentiment",
                    scale=alt.Scale(domain=domain, range=range_),
                    legend=alt.Legend(orient="right"))
).properties(
    width=800,
    height=500,
).configure_legend(
    orient='top'
)

chart

C:\Users\utente\AppData\Local\Temp\ipykernel_13780\3696713326.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_grouped = df.groupby([df['date'].dt.to_period('M'), 'Sentiment']).size().unstack()


alt.Chart(...)

In [14]:
def extract_year(value):
    return df[df['Year']==value]

y22 = extract_year(2022)
y23 = extract_year(2023)

In [19]:
print ("dimensioni dataset 2022 (righe x colonne): "+ str(y22.shape))
print ("dimensioni dataset 2023 (righe x colonne): "+ str(y23.shape))

dimensioni dataset 2022 (righe x colonne): (77338, 19)
dimensioni dataset 2023 (righe x colonne): (209971, 19)


In [ ]:
#prendo il dataset, cerco le frequenze delle parole per sentimento (=risultato: tre dataframe, uno per sentimento e relativi a un anno.)
#creo un dataset con 2 colonne (uno per ogni sentimento), le colonne sono: word, frequency
#creo una visualizzazione scatter plot con colore abbinato al sentimento corrente
#mostro tutti i puntini usando un Alt che in descrizione riporta il valore label di word
#mostro una legenda con le top 20 più frequenti

In [20]:
#prendo il dataset, cerco le frequenze delle parole per sentimento.
def words_finder(df, sentiment):
    tokens = []
    df = df[(df['Sentiment'] == sentiment)]
    for row in df['tweetText_proc']:
        tokens.extend(word_tokenize(str(row).lower()))  # Tokenization and lower conversion of the strings

    word_freq = FreqDist(tokens)
    sorted_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    banned_w = ['chatgpt', 'ai', ' ai', 'ai ', 'gpt', 'openai', 'artificialintelligence', 'amp', 'https'] #words to filter out
    sorted_freq = [(word, freq) for word, freq in sorted_freq if word not in banned_w]
    return sorted_freq #frequencies sorted for the choosen sentiment

def createDffreqs(sorted_freq): #ritorna un dataframe con una colonna contenente la parola ed una con la sua frequenza assoluta
    freq_df = pd.DataFrame(sorted_freq, columns=['word', 'frequency'])
    freq_df.sort_values(by='frequency', ascending=False, inplace=True)
    return freq_df

In [36]:
y22pos = createDffreqs(words_finder(y22, 'pos')) #dataframe 
top22pos = y22pos.head(20)

In [37]:
y22neg = createDffreqs(words_finder(y22, 'neg')) #dataframe 
top22neg = y22neg.head(20)

In [38]:
y22neu = createDffreqs(words_finder(y22, 'neu')) #dataframe 
top22neu = y22neu.head(20)

In [71]:
# Concatenate the three DataFrames into a single DataFrame
df22words = pd.concat([top22pos, top22neg, top22neu], keys=['Positive', 'Negative', 'Neutral'])
options = ['Positive', 'Negative', 'Neutral']
labels = [option + ' ' for option in options]

# Define color domain and range
domain = ['pos', 'neg', 'neu']
range_ = ['green', 'red', 'blue']

input_dropdown = alt.binding_radio(
    # Add the empty selection which shows all when clicked
    options=options + [None],
    labels=labels + ['All'],
    name='Sentiment: '
)

selection = alt.selection_point(
    fields=['Sentiment'],
    bind=input_dropdown,
)
# Hide the y-axis
scatter_plot = scatter_plot.configure_axisY(
    ticks=False,
    labels=False
)

# Hide the y-axis
scatter_plot = scatter_plot.configure_axisY(
    ticks=False,
    labels=False
)
# Reset the index of the concatenated DataFrame
df22words = df22words.reset_index(level=0).rename(columns={'level_0': 'Sentiment'})
alt.data_transformers.disable_max_rows()
alt.Chart(df).mark_point(filled=True).encode(
    alt.X('word'),
    alt.Y('frequency'),
    alt.Size('frequency'),
    alt.OpacityValue(0.7),
    alt.Color('Sentiment').scale(domain=options, range=range_),
    tooltip = [alt.Tooltip('word:N'),
               alt.Tooltip('frequency:Q'),
               alt.Tooltip('frequency:Q'),
               #alt.Tooltip('Sentiment:Q')
              ]
).add_params(
    selection
).transform_filter(
    selection
).properties(
    width=600,  # Set the width of the chart to your desired value
    height=400  # Set the height of the chart to your desired value
)

alt.Chart(...)